In [16]:
import customtkinter as ctk
import requests
from tkinter import messagebox
from datetime import datetime
from io import BytesIO
from PIL import Image, ImageTk

In [17]:
API = "http://127.0.0.1:8000"

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

In [ ]:
#GUUUUI
class App(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Учет заявок на ремонт бытовой техники")  
        self.geometry("1000x650")
        self.user = None
        self.last_status = {}

        self.auth_frame = AuthFrame(self, on_login=self.on_login)
        self.auth_frame.pack(fill="both", expand=True)

        self.main_frame = None

    def on_login(self, user_dict: dict):
        self.user = user_dict
        self.auth_frame.pack_forget()
        self.auth_frame.destroy()
        self.main_frame = MainFrame(self, user=self.user, on_logout=self.on_logout)
        self.main_frame.pack(fill="both", expand=True)

    def on_logout(self):
        self.user = None
        if self.main_frame:
            self.main_frame.pack_forget()
            self.main_frame.destroy()
            self.main_frame = None
        self.auth_frame = AuthFrame(self, on_login=self.on_login)
        self.auth_frame.pack(fill="both", expand=True)


class AuthFrame(ctk.CTkFrame):
    def __init__(self, master, on_login):
        super().__init__(master)
        self.on_login = on_login

        title = ctk.CTkLabel(
            self,
            text="Сервисный центр — вход в систему",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        title.pack(pady=20)

        tabs = ctk.CTkTabview(self, width=420, height=340)
        tabs.pack(pady=10)

        t_login = tabs.add("Вход")
        t_reg = tabs.add("Регистрация")

       
        self.login_e = ctk.CTkEntry(t_login, placeholder_text="Логин")
        self.pass_e = ctk.CTkEntry(t_login, placeholder_text="Пароль", show="*")
        btn = ctk.CTkButton(t_login, text="Войти", command=self.do_login)

        self.login_e.pack(pady=10, padx=20, fill="x")
        self.pass_e.pack(pady=10, padx=20, fill="x")
        btn.pack(pady=10)

        
        self.fio_e = ctk.CTkEntry(t_reg, placeholder_text="ФИО")
        self.phone_e = ctk.CTkEntry(t_reg, placeholder_text="Телефон")
        self.rlogin_e = ctk.CTkEntry(t_reg, placeholder_text="Логин")
        self.rpass_e = ctk.CTkEntry(t_reg, placeholder_text="Пароль", show="*")
        rbtn = ctk.CTkButton(t_reg, text="Зарегистрироваться", command=self.do_register)

        for w in (self.fio_e, self.phone_e, self.rlogin_e, self.rpass_e):
            w.pack(pady=8, padx=20, fill="x")
        rbtn.pack(pady=10)

    def do_login(self):
        try:
            r = requests.post(
                f"{API}/login",
                json={"login": self.login_e.get(), "password": self.pass_e.get()},
                timeout=5
            )
            if r.status_code != 200:
                detail = "Неверный логин или пароль"
                try:
                    detail = r.json().get("detail", detail)
                except:
                    pass
                messagebox.showerror("Ошибка входа", detail)
                return
            self.on_login(r.json())
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось подключиться к серверу.\n{e}")

    def do_register(self):
        try:
            payload = {
                "fio": self.fio_e.get(),
                "phone": self.phone_e.get(),
                "login": self.rlogin_e.get(),
                "password": self.rpass_e.get()
            }
            r = requests.post(f"{API}/register", json=payload, timeout=5)
            if r.status_code != 200:
                detail = "Не удалось зарегистрироваться"
                try:
                    detail = r.json().get("detail", detail)
                except:
                    pass
                messagebox.showerror("Ошибка регистрации", detail)
                return
            messagebox.showinfo("Готово", "Регистрация успешна. Теперь войдите.")
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось подключиться к серверу.\n{e}")


class MainFrame(ctk.CTkFrame):
    def __init__(self, master, user: dict, on_logout):
        super().__init__(master)
        self.user = user
        self.on_logout = on_logout

        
        top = ctk.CTkFrame(self)
        top.pack(fill="x", padx=10, pady=10)

        self.user_label = ctk.CTkLabel(
            top,
            text=f"Пользователь: {user.get('fio')} | роль: {user.get('role')}",
            font=ctk.CTkFont(size=14, weight="bold")
        )
        self.user_label.pack(side="left", padx=10)

        logout_btn = ctk.CTkButton(top, text="Выйти", command=self.on_logout)
        logout_btn.pack(side="right", padx=10)

        self.tabs = ctk.CTkTabview(self)
        self.tabs.pack(fill="both", expand=True, padx=10, pady=10)

        role = user.get("role")

        
        self.tab_requests = self.tabs.add("Заявки")
        self.build_requests_tab()

        
        if role in ("client", "operator", "admin"):
            self.tab_create = self.tabs.add("Новая заявка")
            self.build_create_tab()

        
        if role in ("specialist", "operator", "manager", "admin"):
            self.tab_comments = self.tabs.add("Комментарии")
            self.build_comments_tab()

        
        if role in ("manager", "admin"):
            self.tab_extend = self.tabs.add("Продление")
            self.build_extend_tab()

        
        if role in ("operator", "manager", "admin"):
            self.tab_edit = self.tabs.add("Редактирование")
            self.build_edit_tab()

        
        if role in ("operator", "manager", "admin"):
            self.tab_stats = self.tabs.add("Статистика")
            self.build_stats_tab()

        
        self.after(1200, self.poll_status_changes)

    
    def headers(self):
        return {"X-User-Id": str(self.user["id"])}

    def api_get(self, path):
        return requests.get(f"{API}{path}", headers=self.headers(), timeout=8)

    def api_post(self, path, json):
        return requests.post(f"{API}{path}", headers=self.headers(), json=json, timeout=8)

    def api_put(self, path, json):
        return requests.put(f"{API}{path}", headers=self.headers(), json=json, timeout=8)

    
    def build_requests_tab(self):
        bar = ctk.CTkFrame(self.tab_requests)
        bar.pack(fill="x", pady=5)

        self.search_e = ctk.CTkEntry(bar, placeholder_text="Поиск (номер/тип/модель/описание)")
        self.search_e.pack(side="left", padx=5, fill="x", expand=True)

        ctk.CTkButton(bar, text="Найти", command=self.search_requests).pack(side="left", padx=5)
        ctk.CTkButton(bar, text="Обновить", command=self.load_requests).pack(side="left", padx=5)

        self.req_text = ctk.CTkTextbox(self.tab_requests)
        self.req_text.pack(fill="both", expand=True, padx=5, pady=5)

        self.load_requests()

    def render_requests(self, items):
        self.req_text.delete("1.0", "end")
        for r in items:
            start_date = r.get("start_date") or "—"
            due_date = r.get("due_date") or "—"
            line = (
                f"ID:{r['id']} | {r.get('status','—')} | {r.get('climate_tech_type','—')} | {r.get('climate_tech_model','—')}\n"
                f"  Дата добавления: {start_date} | Срок (продлеваемый): {due_date}\n"
                f"  Проблема: {r.get('problem_description','—')}\n"
                f"  Клиент: {r.get('client_fio') or '—'} {r.get('client_phone') or ''} | Мастер: {r.get('master_fio') or '—'}\n"
                f"{'-'*90}\n"
            )
            self.req_text.insert("end", line)

    def load_requests(self):
        try:
            r = self.api_get("/requests")
            if r.status_code != 200:
                messagebox.showerror("Ошибка", "Не удалось получить заявки")
                return
            self.render_requests(r.json())
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def search_requests(self):
        q = self.search_e.get().strip()
        if not q:
            self.load_requests()
            return
        try:
            r = self.api_get(f"/requests/search?q={q}")
            if r.status_code != 200:
                messagebox.showerror("Ошибка", "Поиск не выполнен")
                return
            self.render_requests(r.json())
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    
    def build_create_tab(self):
        box = ctk.CTkFrame(self.tab_create)
        box.pack(padx=10, pady=10, fill="x")

        self.type_e = ctk.CTkEntry(box, placeholder_text="Тип техники (например: Холодильник)")
        self.model_e = ctk.CTkEntry(box, placeholder_text="Модель")
        self.problem_e = ctk.CTkEntry(box, placeholder_text="Описание проблемы")

        for w in (self.type_e, self.model_e, self.problem_e):
            w.pack(pady=8, fill="x")

        ctk.CTkButton(box, text="Создать заявку", command=self.create_request).pack(pady=10)

        ctk.CTkLabel(
            self.tab_create,
            text="Дата добавления формируется автоматически.",
            text_color="#cfcfcf"
        ).pack(pady=5)

    def create_request(self):
        try:
            payload = {
                "climate_tech_type": self.type_e.get(),
                "climate_tech_model": self.model_e.get(),
                "problem_description": self.problem_e.get()
            }
            r = self.api_post("/requests", payload)
            if r.status_code != 200:
                messagebox.showerror("Ошибка", "Не удалось создать заявку")
                return
            messagebox.showinfo("Готово", f"Заявка создана (ID: {r.json().get('request_id')})")
            self.load_requests()
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    
    def build_comments_tab(self):
        box = ctk.CTkFrame(self.tab_comments)
        box.pack(padx=10, pady=10, fill="x")

        self.c_req_id = ctk.CTkEntry(box, placeholder_text="ID заявки")
        self.c_msg = ctk.CTkEntry(box, placeholder_text="Комментарий")
        for w in (self.c_req_id, self.c_msg):
            w.pack(pady=8, fill="x")

        ctk.CTkButton(box, text="Добавить комментарий", command=self.add_comment).pack(pady=10)

    def add_comment(self):
        try:
            payload = {"request_id": int(self.c_req_id.get()), "message": self.c_msg.get()}
            r = self.api_post("/comments", payload)
            if r.status_code != 200:
                messagebox.showerror("Ошибка", "Не удалось добавить комментарий")
                return
            messagebox.showinfo("Готово", "Комментарий добавлен")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    
    def build_extend_tab(self):
        box = ctk.CTkFrame(self.tab_extend)
        box.pack(padx=10, pady=10, fill="x")

        self.e_req_id = ctk.CTkEntry(box, placeholder_text="ID заявки")
        self.e_date = ctk.CTkEntry(box, placeholder_text="Новая дата срока (ГГГГ-ММ-ДД)")

        self.e_req_id.pack(pady=8, fill="x")
        self.e_date.pack(pady=8, fill="x")

        ctk.CTkButton(box, text="Продлить срок", command=self.extend_deadline).pack(pady=10)

    def extend_deadline(self):
        try:
            payload = {"request_id": int(self.e_req_id.get()), "new_date": self.e_date.get()}
            r = self.api_put("/requests/extend", payload)
            if r.status_code != 200:
                messagebox.showerror("Ошибка", "Не удалось продлить срок")
                return
            messagebox.showinfo("Готово", "Срок продлён")
            self.load_requests()
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    
    def build_edit_tab(self):
        box = ctk.CTkFrame(self.tab_edit)
        box.pack(padx=10, pady=10, fill="x")

        self.u_req_id = ctk.CTkEntry(box, placeholder_text="ID заявки")
        self.u_status = ctk.CTkEntry(box, placeholder_text="Статус (Новая заявка / В процессе / Готова к выдаче)")
        self.u_problem = ctk.CTkEntry(box, placeholder_text="Новое описание проблемы (если нужно)")
        self.u_master = ctk.CTkEntry(box, placeholder_text="ID мастера (master_id)")

        for w in (self.u_req_id, self.u_status, self.u_problem, self.u_master):
            w.pack(pady=7, fill="x")

        ctk.CTkButton(box, text="Сохранить изменения", command=self.update_request).pack(pady=10)

    def update_request(self):
        try:
            rid = int(self.u_req_id.get())
            payload = {}

            if self.u_status.get().strip():
                payload["status"] = self.u_status.get().strip()
            if self.u_problem.get().strip():
                payload["problem_description"] = self.u_problem.get().strip()
            if self.u_master.get().strip():
                payload["master_id"] = int(self.u_master.get().strip())

            if not payload:
                messagebox.showwarning("Внимание", "Нечего сохранять (поля пустые).")
                return

            r = self.api_put(f"/requests/{rid}", payload)
            if r.status_code != 200:
                messagebox.showerror("Ошибка", "Не удалось обновить заявку")
                return

            messagebox.showinfo("Готово", "Заявка обновлена")
            self.load_requests()
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

   
    def build_stats_tab(self):
        top = ctk.CTkFrame(self.tab_stats)
        top.pack(fill="x", pady=5)
        ctk.CTkButton(top, text="Обновить статистику", command=self.load_stats).pack(side="left", padx=5)

        self.stats_text = ctk.CTkTextbox(self.tab_stats)
        self.stats_text.pack(fill="both", expand=True, padx=5, pady=5)

        self.load_stats()

    def load_stats(self):
        try:
            r = self.api_get("/stats")
            if r.status_code != 200:
                messagebox.showerror("Ошибка", "Не удалось получить статистику")
                return

            s = r.json()
            self.stats_text.delete("1.0", "end")
            self.stats_text.insert("end", f"Выполнено заявок: {s.get('done_count', 0)}\n")
            self.stats_text.insert("end", f"Среднее время выполнения (дней): {s.get('avg_days', 0):.2f}\n")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    
    def poll_status_changes(self):
        try:
            r = self.api_get("/requests")
            if r.status_code == 200:
                items = r.json()
                for it in items:
                    rid = it["id"]
                    st = it.get("status")
                    old = self.master.last_status.get(rid)
                    if old is None:
                        self.master.last_status[rid] = st
                    elif old != st:
                        self.master.last_status[rid] = st
                        messagebox.showinfo("Уведомление", f"Статус заявки #{rid} изменился: {old} → {st}")
        except:
            pass
        self.after(5000, self.poll_status_changes)


if __name__ == "__main__":
    app = App()
    app.mainloop()